In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Usado para o método split: é possível fazer um split manualmente, mas train_test_split é mais eficiente e há estratificação já implementada.
from sklearn.model_selection import train_test_split

In [18]:
# Realiza alguns imports para a especificação de tipos
from typing import Self, Optional

In [ ]:
class Dataset:
    def __init__( self, data: pd.DataFrame, *, label_column : int = -1, column_names : Optional[list[str]] = None ) -> None:
        """
        Inicializa a classe Dataset.

        Args:
            data: DataFrame com os dados do conjunto de dados.
            label_column: Posição da coluna de rótulo (por padrão é -1, última coluna).
            column_names: Lista opcional com os nomes das colunas.
        """

        self.data = data                    # Guarda o DataFrame do conjunto de dados
        self.label_column = label_column    # Armazena o índice para a coluna de rótulo
        self.column_names = column_names    # Armazena a lista com os rótulos das colunas do DataFrame

        # Se for especificado, define o nome das colunas
        if self.column_names is not None:
            self._set_column_names()

    def _set_column_names( self ) -> None:
        """
        Atualiza os rótulos das colunas do DataFrame, conforme o valor no atributo column_names.
        """

        self.data.columns = self.column_names

    def shuffle( self, random_state: Optional[int] = None ) -> Self:
        """
        Retorna uma nova instância da classe com os dados embaralhados.

        Args:
            random_state: Semente opcional para reprodução de resultados.
        
        Returns:
            Uma nova instância do Dataset com os dados embaralhados.
        """

        # Obtém um DataFrame embaralhado e com os índices resetados
        data = self.data.sample(frac=1, random_state=random_state).reset_index( drop=True )

        # Retorna uma nova instância da classe
        return self.__class__( data, label_column = self.label_column, column_names = self.column_names )
    
    def split( self, train_size: int = 0.8, *, random_state: Optional[int] = None, stratify: bool = False ) -> tuple[Self, Self]:
        """
        Divide o Dataset em conjunto de treinamento e de conjunto de teste.

        Args:
            train_size: Proporção do conjunto de treinamento (0-1).
            random_state: Semente opcional para reprodução de resultados.
            stratify: Se for True, manterá a proporção entre as classes
        
        Returns:
            Tupla( train_set, test_set )
        """

        # Verifica se stratify foi setado
        _stratify = self.y if stratify == True else None

        # Separa o conjunto de dados em dois
        train, test = train_test_split(
            self.data,
            train_size = train_size,
            random_state = random_state,
            stratify = _stratify
        )

        # Retorna as instâncias para Datasets dos conjuntos separados
        return (
            self.__class__( train, label_column = self.label_column, column_names = self.column_names ),
            self.__class__( test, label_column = self.label_column, column_names = self.column_names )
        )
    
    @classmethod
    def from_file( cls, filepath : str, *, comment_marker : str = "#", missing_marker : str = "?", label_column : int = -1, column_names : Optional[list[str]] = None ) -> None:
        """
        Inicializa a classe a partir de um arquivo CSV.

        Args:
            filepath: Caminho para o arquivo CSV.
            comment_marker: Marcador para linhas de comentário (serão ignoradas).
            missing_marker: Marcador para elementos faltantes (substituídos por NaN).
            Posição da coluna de rótulo (por padrão é -1, última coluna).
            column_names: Lista opcional com o nome das colunas.
        """

        # Lê o arquivo CSV (Não existe linhas de header no arquivo)
        data = pd.read_csv( filepath, header=None, comment=comment_marker, na_values=missing_marker )

        # Remover elementos faltantes
        data = data.dropna()

        # Retorna uma chamada para o construtor normal da classe
        return cls( data, label_column = label_column, column_names = column_names )
    
    @property
    def X( self ) -> pd.DataFrame:
        """ Retorna as colunas de atributos """

        # Caso mais simples, a coluna de label está no final
        if self.label_column == -1:
            return self.data.iloc[:, :self.label_column]
        
        # Se a coluna de resultados estiver em uma posição diferente da coluna final, compõe o DataFrame com os atributos
        return pd.concat([
            self.data.iloc[:, :self.label_column],  
            self.data.iloc[:, self.label_column+1:]
        ], axis=1 )        
    
    @property
    def y( self ) -> pd.Series:
        """ Retorna a coluna de rótulo """

        return self.data.iloc[:, self.label_column]
    
    @property
    def shape( self ) -> tuple[int, int, int]:
        """
        Retorna as dimensões do Dataset

        Returns:
            Tupla[m, n, k] onde:
            m = número de instâncias
            n = número de atributos 
            k = número de classes
        """

        m, n = self.data.shape
        k = len( self.y.unique() )

        # A coluna de rótulo está contada em n
        return (m, n-1, k)
    
    def __len__( self ):
        """ Retorna o número de instâncias do conjunto de dados. """
        return len( self.data )
    
    def __repr__( self ):
        """ Retorna uma string representando o objeto. """
        m, n, k = self.shape
        return f"Dataset(instâncias={m}, features={n}, classes={k})"

In [23]:
path_dataset = r"datasets\iris.data"    # caminho do arquivo
column_names = ["sepal length", "sepal width", "petal length", "petal width", "class"] # Nome dos campos

data = Dataset.from_file( path_dataset, label_column=-1 )
data.split()

(Dataset(instâncias=120, features=4, classes=3),
 Dataset(instâncias=30, features=4, classes=3))